In [1]:
import numpy as np
import glob
import os, re
# import cv2
import random
import matplotlib.pyplot as plt
import datetime
import sklearn.preprocessing
import pydot
import pandas as pd
import math
import time

from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import multiprocessing
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, AveragePooling2D,ZeroPadding2D,Flatten,Dense,Dropout,SpatialDropout2D,concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D,SpatialDropout1D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Multiply, Average, Add, Concatenate
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers,activations

# %tf-nightly_version 2.x
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# tf.config.experimental.list_physical_deWces('GPU')
# deWce_name = tf.test.gpu_deWce_name()

# if deWce_name != '/deWce:GPU:0':
#   raise SystemError('GPU deWce not found')
# print('Found GPU at: {}'.format(deWce_name))

########################################
# Settings for reproducible results
########################################

seed = 1
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)

from tensorflow.keras import backend as BK
#config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.random.set_seed(seed)
#sess = tf.Session(graph=tf.get_default_graph(), config=config)
#BK.set_session(sess)

########################################
# Image resizer
########################################

def rebin(arr, new_shape):
    """Rebin 2D array arr to shape new_shape by maximum."""
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1])
    return arr.reshape(shape).max(-1).max(1)

### Reading in OGLE-III data
The neural network was trained with normed periods and reddening-free (Wesenheit) brightness values.<br>
To be able to use your new inputs, we need to transform them.<br>
For this, you need to run the following block, which will read in the necessary numerical data.

In [2]:
########################################
# Reading in OGLE-III data
########################################

np.random.seed(42)

#img_dir_base = "o3_512_lightcurves_lmc/ogle3_newlc512_"
#var_classes = ['acep','cep','dsct','ecl','rrlyr','t2cep']
#var_classes = ['acep','rrlyr','t2cep']
o3data = pd.read_csv('ogle3_full.dat', delim_whitespace=True,header=None,index_col=0)
o3data.columns = ['i','v','period','amplitude']
o3data.index.name = 'id'

o3data.loc[o3data['i'] == '-', 'i'] = np.nan
o3data.loc[o3data['v'] == '-', 'v'] = np.nan
o3data.loc[o3data['v'] == '99.999', 'v'] = np.nan
o3data.loc[o3data['amplitude'] == '-', 'amplitude'] = np.nan

o3data = o3data.astype('float')

period_full = o3data.loc[(o3data['period'] < 2700) & (o3data['period'] > 0.041),['period']].values
period_full_log = np.log(period_full)
logMin = np.min(period_full_log)
logMax = np.max(period_full_log)


### SETUP
Set the classification for main variable stars only or extend the classifier for subtypes.<br>
The neural network was trained for the following main variable star types:<br>
- Anomalous Cepheids
- Delta Scutiis
- Classical Cepheids
- Eclipsing binaries
- RR Lyrae stars
- Type II Cepheids
<br>
The neural network is capable to identify the following subtypes:<br>
(Change the variable 'maintype' to False)
<br>
- Anomalous Cepheids: F, 1O
- Delta Scutiis: Singlemode, Multimode
- Classical Cepheids: F, 1O, 1O/2O
- Eclipsing binaries: EC, ED, ESD
- RR Lyrae stars: RRab, RRc, RRd
- Type II Cepheids: BL Her, RV Tau, WVir

In [3]:
maintype = True

weightName = 'minn_'

if maintype:
    weightName = weightName + 'maintype_img'
    predictionLabels = ['ACEP', 'CEP', 'DSCT', 'ECL', 'RRLYR', 'T2CEP']
else:
    weightName = weightName + 'subtype_img'
    predictionLabels = ['ACEP 1O', 'ACEP F',
                        'CEP 1O', 'CEP 1O2O', 'CEP F',
                        'DSCT MULTI', 'DSCT SINGL',
                        'ECL EC', 'ECL ED', 'ECL ESD',
                        'RRLYR RRAB', 'RRLYR RRC', 'RRLYR RRD',
                        'T2CEP BLHER', 'T2CEP RVTAU', 'T2CEP WVIR']


### DATA
The Multiple-input Neural Network was trained with different numbers of inputs.<br>
The basic input is a phase-folded lightcurve, every measurement points are plotted with dots (not connected with lines!).<br>
Additional inputs:<br>
- Maint type classifier: period
- Subtpye classifier: period, Wesenheit-index

In [4]:
#
# PHASE-FOLDED LIGHT CURVE(S)
#
img_dir = "../HajduT_lc/" # Directory of the phase folded light curve(s)
fileNames = sorted(glob.glob(img_dir + "/*.png"))

newImgS = 128
newSize = (newImgS,newImgS)

imgsForTest = []

for fn in fileNames:
    img = image.load_img(fn, color_mode="grayscale")
    img = image.img_to_array(img,dtype="uint8")
    img = rebin(img, newSize).reshape(newImgS,newImgS,1)
    imgsForTest.append(img)

imgsForTest = np.array(imgsForTest)

#
# PERIOD(S)
#

periods = []

#
# WESENHEIT-INDEX (REDDENING-FREE BRIGHTNESS)
#

wbrightness = []

### Loading the appropriate weight file

In [5]:
if not periods:
    pweightname = ''
else:
    pweightname = '_per'

if not wbrightness:
    wweightname = ''
else:
    wweightname = '_w'

weightName = weightName + pweightname + wweightname + '.h5'


### RUN THE CLASSIFIER

In [6]:
print('>>> Loading', weightName)
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

model = keras.models.load_model(weightName)
start_time = time.time()

print('>>> Classification in progress...')

pred = model.predict(imgsForTest)

pred = np.around(pred,3)

classifications = pd.DataFrame({'ACep': pred[:, 0],
                                'Cep': pred[:, 1],
                                'DSct': pred[:, 2],
                                'ECL': pred[:, 3],
                                'RRLyr': pred[:, 4],
                                'T2Cep': pred[:, 5]})

classifications['fileName'] = fileNames
classifications = classifications[['fileName', 'ACep', 'Cep', 'DSct', 'ECL', 'RRLyr', 'T2Cep']]

print('+++ Classification done.')
print("+++ Execution took:", round(time.time() - start_time, 2) ,'secs' )


>>> Loading minn_maintype_img.h5
>>> Classification in progress...
+++ Classification done.
+++ Execution took: 1.93 secs


In [7]:
classifications.head(25)

,fileName,ACep,Cep,DSct,ECL,RRLyr,T2Cep
0,../HajduT_lc/028118.png,0.000,0.000,1.000,0.000,0.000,0.000
1,../HajduT_lc/031517.png,0.000,0.000,0.947,0.052,0.000,0.000
2,../HajduT_lc/032146.png,0.000,0.000,0.000,1.000,0.000,0.000
3,../HajduT_lc/032583.png,0.000,0.000,0.949,0.046,0.005,0.000
4,../HajduT_lc/033271.png,0.000,0.000,0.874,0.114,0.010,0.001
5,../HajduT_lc/033287.png,0.000,0.000,0.994,0.006,0.000,0.000
6,../HajduT_lc/033822.png,0.000,0.000,0.999,0.001,0.000,0.000
7,../HajduT_lc/035496.png,0.000,0.000,0.014,0.986,0.000,0.000
8,../HajduT_lc/035645.png,0.000,0.000,1.000,0.000,0.000,0.000
9,../HajduT_lc/035691.png,0.001,0.005,0.180,0.000,0.813,0.002


In [ ]:
classifications.to_csv('classification_results',index=False,sep=',')

In [ ]:

# i = 9
# periodValue = period_full[i]
# periodValue = 0.2502326
# print(periodValue)
# print(np.log(periodValue) )
# print(logMin, logMax)

# periodNormedValue = (np.log(periodValue)-logMin)/(logMax-logMin)
# print(periodNormedValue)